# nbody galaxy lab

Analysis of simulation data...

First, import required modules.

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import animation
from IPython.display import Video

## Get data

Set path to data and file name prefix.

In [ ]:
# Filename prefix
prefix = 'nbody'

# Data directory
directory = pathlib.Path('../data')

Get a list of paths to snapshot files.

In [ ]:
snaps = sorted(directory.glob(f'{prefix}_*.csv'))

See the list of (the first 10) snapshots

In [ ]:
print(snaps[:10])

Get the data for the first snapshot as a "Pandas data frame". Remember Python counts from zero, so we choose `snaps[0]`.

In [ ]:
df = pd.read_csv(snaps[0])

See what's in the file.

In [ ]:
df

Now read all the files into dataframes. Note: if the files were very big, you wouldn't normally read all the data into memory at once.

In [ ]:
dataframes = list()
for snap in snaps:
    dataframes.append(pd.read_csv(snap))

## Plot data

Plot the initial conditions particles in the xy-plane.

In [ ]:
ax = df.plot.scatter('x', 'y', c='k', s=0.5)
ax.set(aspect='equal')

## Make a movie

We generate an animation of the simulation and save it as mp4.

In [ ]:
def ani():

    # First generate a plot of initial conditions
    fig, ax = plt.subplots()
    df = dataframes[0]
    scat = ax.scatter(df['x'], df['y'], c='k', s=0.5)
    ax.set(xlim=(-200, 200), ylim=(-200, 200))

    print('Generating animation... Animating frame:')
    
    # Then a function to update the positions
    def animate(idx):
        print(f'{idx}/{len(dataframes)}', end=', ')
        df = dataframes[idx]
        xy = np.array([df['x'], df['y']]).T
        scat.set_offsets(xy)
        return [scat]


    anim = animation.FuncAnimation(fig, animate, frames=len(dataframes))

    # Adjust the "fps" argument to change the animation speed
    # Adjust the "dpi" argument to change the animation resolution
    anim.save(
        directory / 'animation.mp4', extra_args=['-vcodec', 'libx264'], fps=50, dpi=100
    )
    
    plt.close()

Run animation function to generate movie.

In [ ]:
ani()

Display video.

In [ ]:
Video(directory / 'animation.mp4')

## Plot conserved quantities

In [ ]:
# Load the data
df = pd.read_csv(directory / 'nbody.csv')

# Plot energy
df['total_energy'] = df['kinetic_energy'] + df['potential_energy']
df.plot('time', ['kinetic_energy', 'potential_energy', 'total_energy'])